What is a graph?

What do you want to do on it?


Search a graph
spanning tree
routing on a graph
shortest path
maximum flow
minimum cut
maximum matching
minimum spanning tree
strongly connected components
biconnected components
articulation points
bridges
2-edge connected components


colorings
covers




In [ ]:
V = int
Edge = tuple[V,V]

# flat reps
Graph1 = set[Edge]
Graph2 = list[Edge]
Graph3 = dict[V, set[V]]


# structured representations

In [ ]:
# kerngihan Lin

def kl(edges):
    A = set()
    B = set()
    

